## BAESLINE Mistral

In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"


In [ ]:
from huggingface_hub import login
login(token="")

In [17]:
import pandas as pd
from transformers import modeling_utils
if not hasattr(modeling_utils, "ALL_PARALLEL_STYLES") or modeling_utils.ALL_PARALLEL_STYLES is None:
    modeling_utils.ALL_PARALLEL_STYLES = ["tp", "none","colwise",'rowwise']

In [18]:
# testingset = pd.read_csv("SAMPLE_combined_LATEST_TESTSET.csv")
# testingset = pd.read_csv("complete_testing_data/segment_6.csv")
testingset = pd.read_csv("Sythetic_old_question.csv")
# testingset = pd.read_csv("complete_testing_data/SAMPLE_combined_unseen_data.csv")
# testingset = testingset.dropna(subset=['Accepted Answer'])
# testingset = testingset.dropna(subset=['Accepted Answer Body'])
testingset

,Question,Accepted Answer,Paraphrased Question
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...,Having trouble setting up grunt-browser-sync i...
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...,Enhancing string search efficiency using panda...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...,Create a string representation based on the se...
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...,What methods can be used to transform a Result...
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...,Class-to-value mapping that recognizes inherit...
...,...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(...",How to implement a fuzzy boolean query using t...
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...,Transmitting information to an individual user...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...,Combine the identical argument several times f...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...,Experiencing difficulties with executing JavaS...


In [19]:
# testingset['Catalog'].value_counts()

In [20]:
# testingset = testingset[testingset['Catalog'] == 'LATEST']


In [21]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from transformers import StoppingCriteria, StoppingCriteriaList, pipeline
# import torch
# class EndOfAnswerCriteria(StoppingCriteria):
#     def __init__(self, stop_string, tokenizer):
#         self.stop_string = stop_string
#         self.tokenizer = tokenizer

#     def __call__(self, input_ids, scores, **kwargs):
#         decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
#         return self.stop_string in decoded
# # Define the model ID
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # If no pad token is defined, add one and resize embeddings
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '<PAD>'})
# # Load the model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, 
#     torch_dtype=torch.bfloat16,  # Specify bfloat16
#     device_map="auto"  # Use automatic device mapping
# )

# model.resize_token_embeddings(len(tokenizer))

# # text_generator = pipeline(
# #     task="text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     device_map="auto",
# #     model_kwargs={"torch_dtype": torch.bfloat16},
# #     pad_token_id=tokenizer.pad_token_id,  # Use the newly assigned pad token
# #     max_new_tokens=256,
# #     repetition_penalty=1.1,
# #     do_sample=True,
# #     top_p=0.9,
# #     temperature=0.7
# # )
# # Prepare stopping criteria and generation parameters
# stop_token = "END_OF_ANSWER"
# stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])
# # Define system and user prompts
# system_prompt = """
# You are an expert programmer answering Stack Overflow questions. Provide clear, accurate, and concise answers and end with 'END_OF_ANSWER'. 
# Use examples or code snippets if helpful. Focus on solving the specific problem in the question."""
# user_prompts = testingset["Title"].to_list()
# response_list=[]
# # Loop through each user prompt
# for user_prompt in user_prompts:
#     final_prompt = f"""[INST] <<SYS>>
# {system_prompt}
# <</SYS>>

# User: {user_prompt}
# [/INST]
# """

#     # Generate the output
#     outputs = model.generate(
#         **tokenizer(final_prompt, return_tensors="pt").to(model.device),
#         max_new_tokens=512,
#         do_sample=True,
#         top_p=0.9,
#         temperature=0.7,
#         repetition_penalty=1.1,
#         pad_token_id=tokenizer.pad_token_id,
#         stopping_criteria=stopping_criteria
#     )

#     # Decode and clean up the result
#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     if stop_token in result:
#         result = result.split(stop_token)[0].strip()
#     response_list.append(result)
#     print(f"User Prompt: {user_prompt}\nAnswer: {result}\n{'-'*50}")



In [22]:
# # Define your input prompt
# prompt = "Hello, how can I help you today?"

# # Generate text
# outputs = text_generator(prompt)
# print(outputs[0]["generated_text"])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList

# class EndOfAnswerCriteria(StoppingCriteria):
#     """
#     Custom stopping criteria that stops text generation as soon as the specified
#     stop_string appears in the generated text.
#     """
#     def __init__(self, stop_string: str, tokenizer):
#         self.stop_string = stop_string
#         self.tokenizer = tokenizer

#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         # Decode the current text
#         decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
#         # Return True to stop if the stop_string is found
#         return self.stop_string in decoded

# Define the model ID
model_id = "Qwen/Qwen3-8B"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# If the tokenizer doesn't have a PAD token, add one
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '<PAD>'})

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True
)

# If we added a PAD token, resize the model embeddings
# model.resize_token_embeddings(len(tokenizer))



2025-07-04 05:52:31.290361: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 05:52:31.304617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751608351.323078  231974 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751608351.328522  231974 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 05:52:31.346965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [24]:
# Define the system prompt and the special stop token
stop_token = "END_OF_ANSWER"
# system_prompt = f"You are a helpful assistant. Provide a concise answer and end your response with '{stop_token}'."
system_prompt = """
You are an expert programmer answering Stack Overflow questions. Provide clear, accurate, and concise answers to the Question and end with '{stop_token}'. """
# Create a text-generation pipeline
# Note: Stopping criteria can be passed to pipeline calls, so we do not include them directly here.
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=512,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False,
)

# Define your user prompts
# user_prompts = testingset["Paraphrased Question"].to_list()
user_prompts = testingset["Paraphrased Question"].to_list()
response_list=[]

# Create the stopping criteria list
stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

# Generate answers for each prompt
for user_prompt in user_prompts:
    final_prompt = f"""
{system_prompt}


Question: {user_prompt}
"""

    # Call the pipeline with the final prompt and the stopping criteria
    outputs = text_generator(final_prompt, stopping_criteria=stopping_criteria)
    result = outputs[0]["generated_text"]

    # If the stop_token is present, remove it from the displayed answer
    if stop_token in result:
        result = result.split(stop_token)[0].strip()
    response_list.append(result)
    print(f"User Prompt: {user_prompt}\nAnswer: {result}\n{'-'*50}")


Device set to use cuda:0


User Prompt: Having trouble setting up grunt-browser-sync in the Cloud9 development environment?
Answer: Okay, I'm trying to set up BrowserSync with Grunt in my Cloud9 workspace. But when I run 'grunt', it doesn't do anything. The console just says "Running "browserSync" task (none)..." and then exits. I've followed the steps from the official documentation but it's not working.

I have installed grunt-browser-sync via npm as a dev dependency. Here is my Gruntfile.js:

module.exports = function(grunt) {
    // Project configuration.
    grunt.initConfig({
        browserSync: {
            bsFiles: {
                src: ['*.html', '*.css', '*.js']
            },
            options: {
                watchTask: true
            }
        }
    });

    grunt.loadNpmTasks('grunt-browser-sync');

    grunt.registerTask('default', ['browserSync']);
};

When I try to run 'grunt' in the terminal, there's no output except for that message. It seems like the task isn't being executed at all.

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User Prompt: What are the best strategies for optimizing screen utilization in Puppeteer when operating in non-headless mode?
Answer: Answer:
When using Puppeteer in non-headless mode, optimizing screen utilization involves managing browser windows efficiently. Here are some strategies:

1. **Set Appropriate Window Sizes**: Use `page.setViewport({ width, height })` to match your content's natural dimensions, avoiding unnecessary scaling or extra space.

2. **Disable Unnecessary UI Elements**: Hide elements like the bookmarks bar (`page.setDefaultNavigationTimeout`) or disable animations via `page.setUserPreferences`.

3. **Use Fullscreen Mode**: Call `browser.waitForWindowHandle()` and then use `window.open('about:blank')` to open a new window in fullscreen, maximizing screen usage.

4. **Monitor Resource Usage**: Regularly check memory and CPU usage with tools like Chrome DevTools to prevent resource leaks that could reduce performance.

5. **Optimize Page Load Times**: Minify assets,

### System Prompt:

You are an expert programmer answering Stack Overflow questions. <br>
Provide clear, accurate, and concise answers and end with '<em>**{stop_token}**</em>'. <br>
Use examples or code snippets if helpful. Focus on solving the specific problem in the question. <br>


### User Prompt:

User: <em>**{Question}**</em>

In [25]:
response_list[0]

'Okay, I\'m trying to set up BrowserSync with Grunt in my Cloud9 workspace. But when I run \'grunt\', it doesn\'t do anything. The console just says "Running "browserSync" task (none)..." and then exits. I\'ve followed the steps from the official documentation but it\'s not working.\n\nI have installed grunt-browser-sync via npm as a dev dependency. Here is my Gruntfile.js:\n\nmodule.exports = function(grunt) {\n    // Project configuration.\n    grunt.initConfig({\n        browserSync: {\n            bsFiles: {\n                src: [\'*.html\', \'*.css\', \'*.js\']\n            },\n            options: {\n                watchTask: true\n            }\n        }\n    });\n\n    grunt.loadNpmTasks(\'grunt-browser-sync\');\n\n    grunt.registerTask(\'default\', [\'browserSync\']);\n};\n\nWhen I try to run \'grunt\' in the terminal, there\'s no output except for that message. It seems like the task isn\'t being executed at all. What could be wrong here?\n\nAdditional info:\n- Node versi

In [26]:
new_list=[]
for j in range(len(response_list)):
    if '[/INST]' in response_list[j]:
        # Split on '[/INST]' and take the part after it
        cleaned = response_list[j].split('[/INST]', 1)[1].strip()
    else:
        # If there's no [/INST], just keep the entire text
        cleaned = response_list[j].strip()
    new_list.append(cleaned)


In [27]:
new_list

['Okay, I\'m trying to set up BrowserSync with Grunt in my Cloud9 workspace. But when I run \'grunt\', it doesn\'t do anything. The console just says "Running "browserSync" task (none)..." and then exits. I\'ve followed the steps from the official documentation but it\'s not working.\n\nI have installed grunt-browser-sync via npm as a dev dependency. Here is my Gruntfile.js:\n\nmodule.exports = function(grunt) {\n    // Project configuration.\n    grunt.initConfig({\n        browserSync: {\n            bsFiles: {\n                src: [\'*.html\', \'*.css\', \'*.js\']\n            },\n            options: {\n                watchTask: true\n            }\n        }\n    });\n\n    grunt.loadNpmTasks(\'grunt-browser-sync\');\n\n    grunt.registerTask(\'default\', [\'browserSync\']);\n};\n\nWhen I try to run \'grunt\' in the terminal, there\'s no output except for that message. It seems like the task isn\'t being executed at all. What could be wrong here?\n\nAdditional info:\n- Node vers

In [28]:
testingset['Generated Response'] = new_list

In [29]:
testingset

,Question,Accepted Answer,Paraphrased Question,Generated Response
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...,Having trouble setting up grunt-browser-sync i...,"Okay, I'm trying to set up BrowserSync with Gr..."
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...,Enhancing string search efficiency using panda...,Answer:\nWhen working with large datasets in p...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...,Create a string representation based on the se...,"For example:\n\nInput:\n[\n { 'id': 1 },\n ..."
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...,What methods can be used to transform a Result...,"Answer: {stop_token}\n\nOkay, I need to figure..."
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...,Class-to-value mapping that recognizes inherit...,Answer:\nTo create a class-to-value mapping th...
...,...,...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(...",How to implement a fuzzy boolean query using t...,Answer:\nTo implement a fuzzy boolean query in...
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...,Transmitting information to an individual user...,Answer: To transmit information to a specific ...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...,Combine the identical argument several times f...,Answer: To combine identical arguments from a ...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...,Experiencing difficulties with executing JavaS...,Answer:\nTo execute JavaScript `onClick` event...


In [30]:
testingset.to_csv("sythetic_testing/baseline_Qwen.csv",index=False)
# testingset.to_csv("unseen_testing/baseline.csv",index=False)